In [1]:
import sys, json, itertools
from pathlib import Path
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader, random_split,TensorDataset
import numpy as np
from transformers import T5Tokenizer, T5Config, T5ForConditionalGeneration, T5Model
import pytorch_lightning as pl
tokenizer = T5Tokenizer.from_pretrained('t5-small')

In [2]:
sys.path.append('/home/ubuntu/pycharm/recipe_parsing/src')
from recipe_prep import *
from recipe_t5 import *

In [7]:
vocab, token_types = t5_extra_vocab(31000)
idx2vocab={v:k for k,v in vocab.items()}
model = T5Recipe.from_pretrained('/home/ubuntu/recipe_model')
# model = T5ForConditionalGeneration.from_pretrained('/home/ubuntu/recipe_model')

In [8]:
txt = "Preheat oven to 350 "
input_ids = tokenizer.encode(txt, return_tensors='pt')
input_ids

tensor([[  276, 22777,  4836,    12, 10239,     1]])

In [25]:
triplet_types = (token_types['commands'],
token_types['args']+token_types['resources'],
token_types['resources'])

In [28]:
min_length=10
type_constraints = list(itertools.chain(*itertools.repeat(triplet_types,min_length)))
model.set_generation_constraints(type_constraints)
output_ids = model.generate(input_ids=input_ids, min_length=min_length)
output_ids

BOOM


/home/ubuntu/pycharm/recipe_parsing/src/recipe_t5.py:32: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  ret.append((self.constraints[i] * F.softmax(score)).argmax().item())


[31000,
 31017,
 31014,
 31002,
 31143,
 31014,
 31002,
 31143,
 31014,
 31002,
 31143,
 31014,
 31002,
 31143,
 31014,
 31002,
 31143,
 31014,
 31002]

In [29]:
list(map(idx2vocab.get, output_ids))

['PUT',
 'OVEN_MED',
 'STOVE_MEDHI',
 'USE',
 'TBOWL',
 'STOVE_MEDHI',
 'USE',
 'TBOWL',
 'STOVE_MEDHI',
 'USE',
 'TBOWL',
 'STOVE_MEDHI',
 'USE',
 'TBOWL',
 'STOVE_MEDHI',
 'USE',
 'TBOWL',
 'STOVE_MEDHI',
 'USE']